In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras

In [3]:
#Relation between xs and ys is 2x-1

xs = np.array([-1.0,0.0,2.0,3.0,4.0],dtype=float)
ys = np.array([-3.0,-1.0,3.0,5.0,7.0],dtype=float)

In [4]:
xs

array([-1.,  0.,  2.,  3.,  4.])

#**Model Building**

In [5]:
model = tf.keras.Sequential([keras.layers.Dense(units=1,input_shape=[1])])
model.compile(optimizer='sgd',loss='mean_squared_error')
model.fit(xs,ys,epochs=100,verbose=0)
print(model.predict([10.0]))

[[18.23743]]


#**Custom Loss Function**

In [6]:
def my_huber_loss(y_true,y_pred):
  threshold = 1
  error=y_true - y_pred
  is_small_error=tf.abs(error)< threshold
  small_error_loss = tf.square(error)/2
  big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
  return tf.where(is_small_error,small_error_loss,big_error_loss)

In [7]:
model = tf.keras.Sequential([keras.layers.Dense(units=1,input_shape=[1])])
model.compile(optimizer='sgd',loss=my_huber_loss)
model.fit(xs,ys,epochs=100,verbose=0)
print(model.predict([10.0]))

[[17.85324]]


In [8]:
#wrap custom fun in a function so as to remove the hardcore value of custom function
def my_huber_loss_with_threshold(threshold):
  def my_huber_loss(y_true,y_pred):
    threshold = 1
    error=y_true - y_pred
    is_small_error=tf.abs(error)< threshold
    small_error_loss = tf.square(error)/2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error,small_error_loss,big_error_loss)
  return my_huber_loss

In [9]:
model = tf.keras.Sequential([keras.layers.Dense(units=1,input_shape=[1])])
model.compile(optimizer='sgd',loss=my_huber_loss_with_threshold(threshold=1.2))
model.fit(xs,ys,epochs=100,verbose=0)
print(model.predict([10.0]))

[[16.850943]]


#**Putting Custom Function in class and use that class in a model** 

In [10]:
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):
  threshold = 1
  def __init__(self,threshold):
    supe().__init__()
    self.threshold = threshold

  def call(self,y_true,y_pred):
    error=y_true - y_pred
    is_small_error=tf.abs(error)<= self.threshold
    small_error_loss = tf.square(error)/2
    big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
    return tf.where(is_small_error,small_error_loss,big_error_loss)


In [11]:
model = tf.keras.Sequential([keras.layers.Dense(units=1,input_shape=[1])])
model.compile(optimizer='sgd',loss=my_huber_loss_with_threshold(threshold=1.2))
model.fit(xs,ys,epochs=100,verbose=0)
print(model.predict([10.0]))

[[9.912713]]
